# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fbprophet import Prophet


In [ ]:
df = pd.read_csv('../../data/raw/zillow_data.csv')

In [ ]:
df.head(6)

In [ ]:
def one_zipcode(df, index):
    """This function pulls the data for one zipcode at a time and retuns a DataFrame for using in Prophet."""
    series = df.iloc[index]
    series_data = series.iloc[7:]
    df_series = pd.DataFrame(series_data.values, index = series_data.index, columns = ['y'])
    df_series.index = pd.to_datetime(df_series.index, yearfirst = True, format = '%Y-%m')
    df_series['ds'] = df_series.index
    df_series.reset_index(drop = True, inplace = True)
    df_series['y'] = df_series['y'].astype('int64')
    return df_series

In [ ]:
one_zipcode(df, 5)

In [ ]:
#Need helper function for zipcode
def get_zipcode(df, index):
    """This function extracts the zipcode from the dataframe at the index. Useful in constructing data dictionanry
    later."""
    zipcode = df['RegionName'].iloc[index]
    return zipcode

In [ ]:
get_zipcode(df, 5)

In [ ]:
#pull first zip code from df
series_84654 = df.iloc[0]

In [ ]:
#drop the meta data for the zip code
df_84654_data = series_84654.iloc[7:]

In [ ]:
#take the series that no longer has the first 7 lines, and put it in a DF
df_84654 = pd.DataFrame(df_84654_data.values, index=df_84654_data.index, columns=['y'] )

In [ ]:
#set index as date time format
df_84654.index = pd.to_datetime(df_84654.index,yearfirst=True, format='%Y-%m')

In [ ]:
df_84654['ds']=df_84654.index

In [ ]:
df_84654.reset_index(drop=True, inplace=True)

In [ ]:
df_84654['y'] = df_84654['y'].astype('int64')

In [ ]:
df_84654.y.dtype

In [ ]:
def Prophet_analysis(df):
    """This function instantiates a Prophet model, fits it to the DataFrame, and predicts values which are returned in 
    a forecast Dataframe."""
    m = Prophet(seasonality_mode='multiplicative', interval_width=0.95)
    m.fit(df)
    future = m.make_future_dataframe(60, freq = 'M')
    forecast = m.predict(future)
    return forecast

In [ ]:
Prophet_analysis(one_zipcode(df, 5))

In [ ]:
m = Prophet(seasonality_mode='multiplicative', interval_width=0.95)
m.fit(df_84654)

In [ ]:
future = m.make_future_dataframe(60, freq='M')
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)


In [ ]:
df_84654

In [ ]:
def most_recent_value(df):
    '''
    Finds the most recent median home value for use in future percent calculation.
    '''
    i = -1
    while np.isnan(df.iloc[i]['y']):
        i -= 1

    return df.iloc[i]['y']

In [ ]:
def combine_dataframes(df, forecast):
    """This function combines the original dataframe with data from one zipcode and the forecasted values from the
    Prophet analysis into a single DataFrame. It also adds new columns for percent change from the last value in
    the original dataframe and the estimated gross profit."""
    forecast.rename(columns={'yhat': 'y'}, inplace = True)
    forecast_subset = forecast[forecast['ds']>'2018-04-01'][['ds','y', 'yhat_lower', 'yhat_upper']]
    df_appended = df.append(forecast_subset, sort = True)
    df_appended['pct_change'] = ((df_appended['y']-most_recent_value(df))/most_recent_value(df))*100
    df_appended['gross_profit'] = (df_appended['y']-most_recent_value(df))
    return df_appended

In [ ]:
combine_dataframes(one_zipcode(df, 5), Prophet_analysis(one_zipcode(df, 5)))

In [ ]:
forecast.rename(columns={"yhat": "y"}, inplace = True)

In [ ]:
forecast_subset = forecast[forecast['ds']>'2018-04-01'][['ds','y']]

In [ ]:
df_84654_appended = df_84654.append(forecast_subset)

In [ ]:
df_84654_appended['pct_change'] = ((df_84654_appended['y']-most_recent_value(df_84654))/most_recent_value(df_84654))*100


In [ ]:
df_84654_appended['net_profit'] = (df_84654_appended['y']-most_recent_value(df_84654))


In [ ]:
df_84654_appended

In [ ]:
def make_dict(df, zipcode):
    """This function returns a summary dictionary from the combined dataframe for a given zipcode. This dictionary
    can be appended to a list that will be used to make a final dataframe."""
    pct_change_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['pct_change'].values[0]
    gross_profit_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['gross_profit'].values[0]
    pct_change_5year = df.loc[(df['ds']>='2023-03-31')]['pct_change'].values[0]
    gross_profit_5year = df.loc[(df['ds']>='2023-03-31')]['gross_profit'].values[0]
    estimate_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['y'].values[0]
    upper_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_upper'].values[0]
    lower_1year = df.loc[(df['ds']>'2019-03-31') 
                                       & (df['ds']<'2019-05-01')]['yhat_lower'].values[0]
    estimate_5year = df.loc[(df['ds']>='2023-03-31')]['y'].values[0]
    upper_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_upper'].values[0]
    lower_5year = df.loc[(df['ds']>='2023-03-31')]['yhat_lower'].values[0]
    zipcode_dict = {'Zipcode': zipcode, 
                    'estimate_1year': estimate_1year,
                    'upper_1year': upper_1year,
                    'lower_1year': lower_1year,
                    'pct_change_1year': pct_change_1year,
                    'estimate_5year': estimate_5year,
                    'upper_5year': upper_5year,
                    'lower_5year': lower_5year,
                    'gross_profit_1year': gross_profit_1year, 
                    'pct_change_5year': pct_change_5year, 
                    'gross_profit_5year': gross_profit_5year}
    return zipcode_dict

In [ ]:
make_dict(combine_dataframes(one_zipcode(df, 5), Prophet_analysis(one_zipcode(df, 5))), get_zipcode(df, 5))

In [ ]:
dict1 = [{'Zipcode': series_84654['RegionName'], 
 'pct_change_1year': pct_change_1year, 
 'net_profit_1year': net_profit_1year, 
 'pct_change_5year': pct_change_5year, 
 'net_profit_5year': net_profit_5year},
         {'Zipcode': series_84654['RegionName'], 
 'pct_change_1year': pct_change_1year, 
 'net_profit_1year': net_profit_1year, 
 'pct_change_5year': pct_change_5year, 
 'net_profit_5year': net_profit_5year}]

In [ ]:
pd.DataFrame.from_dict(dict1)

In [ ]:
df_84654_appended.iloc[df_84654_appended[(df_84654_appended['ds']>'2019-03-31') 
                                         & (df_84654_appended['ds']<'2019-05-01')].index]['pct_change'].iloc[0]

In [ ]:
pct_change_1year = df_84654_appended.loc[(df_84654_appended['ds']>'2019-03-31') 
                                       & (df_84654_appended['ds']<'2019-05-01')]['pct_change'].values[0]

In [ ]:
net_profit_1year = df_84654_appended.loc[(df_84654_appended['ds']>'2019-03-31') 
                                       & (df_84654_appended['ds']<'2019-05-01')]['net_profit'].values[0]

In [ ]:
pct_change_5year = df_84654_appended.loc[(df_84654_appended['ds']>='2023-03-31') 
                                       ]['pct_change'].values[0]

In [ ]:
net_profit_5year = df_84654_appended.loc[(df_84654_appended['ds']>='2023-03-31') 
                                       ]['net_profit'].values[0]

In [ ]:
temp.iloc[0]

In [ ]:
one_year_df = pd.DataFrame(columns=['RegionID','RegionName','City','State','Metro','CountyName','y','pct_change','net_profit'])

In [ ]:
ind_row = pd.DataFrame(df_84654_appended[(df_84654_appended['ds']>'2019-03-31') & (df_84654_appended['ds']<'2019-05-01') ][['y','pct_change','net_profit']])
    

In [ ]:
one_year_df = one_year_df.append(df_84654_appended[(df_84654_appended['ds']>'2019-03-31') & (df_84654_appended['ds']<'2019-05-01') ][['y','pct_change','net_profit']]
    )

In [ ]:
one_year_df

In [ ]:
one_year_df.join(pd.DataFrame(series_84654), how='left')

In [ ]:
one_year_df

In [ ]:
series_84654[:7]

# Step 2: Data Preprocessing

In [ ]:
df_84654

In [ ]:
def get_datetimes(df):
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

# Step 6: Interpreting Results